In [3]:
# Needs iqm-benchmarks from the github repo to access all the mGST functions: https://github.com/iqm-finland/iqm-benchmarks
from mGST.low_level_jit import dK, objf
from mGST.low_level_jit import cost_function_jax, cost_function_jax_jit
from iqm.benchmarks.compressive_gst.compressive_gst import GSTConfiguration
from mGST.utility_functions_comparisons import get_full_mgst_parameters_from_configuration
# Check compilation time for contract_jax_jit.
import jax
import numpy as np

backend = "iqmfakeapollo"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1GST

In [4]:
# First using the JAX compilation with smaller qubits to see if this helps
Q1_GST = GSTConfiguration(
    qubit_layouts=[[0]],
    gate_set="1QXYI",
    num_circuits=100,
    shots=1000,
    rank=4,
)

K, X, E, rho, y, J, l, d, pdim, r, n_povm, bsize, meas_samples, n, nt, rK = get_full_mgst_parameters_from_configuration(
    Q1_GST, backend
)

2025-01-14 15:29:54,585 - iqm.benchmarks.logging_config - INFO - Now generating 100 random GST circuits...
2025-01-14 15:29:54,812 - iqm.benchmarks.logging_config - INFO - Will transpile all 100 circuits according to fixed physical layout
2025-01-14 15:29:54,812 - iqm.benchmarks.logging_config - INFO - Transpiling for backend IQMFakeApolloBackend with optimization level 0, sabre routing method all circuits
2025-01-14 15:29:55,299 - iqm.benchmarks.logging_config - INFO - Submitting batch with 100 circuits corresponding to qubits [0]
2025-01-14 15:29:55,307 - iqm.benchmarks.logging_config - INFO - Now executing the corresponding circuit batch
2025-01-14 15:29:55,343 - iqm.benchmarks.logging_config - INFO - Retrieving all counts


In [5]:
# Numba
%timeit dK(X, K, E, rho, J, y, d, r, rK)

5.86 ms ± 264 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
